### Download Dataset
. means delimiter between triples  
?x is variable  
whitespace is significant here as angle brackets have been removed

In [8]:
import re

import pandas as pd

target_dataset = "test"

splits = {'train': 'train.json', 'test': 'test.json'}
df = pd.read_json("hf://datasets/casey-martin/vquanda/" + splits[target_dataset])

In [9]:
df

,uid,question,verbalized_answer,query
0,3986,Count the number of people became famous for w...,There are [8] people known for works commanded...,SELECT DISTINCT COUNT(?uri) WHERE { ?x <http:/...
1,2262,Which location city of Denver Broncos is the p...,The location city of Denver Broncos and place ...,SELECT DISTINCT ?uri WHERE { <http://dbpedia.o...
2,855,Which people are known for appearing on the te...,The people known for Dragons' Den series are [...,SELECT DISTINCT ?uri WHERE {?uri <http://dbped...
3,377,Which office holder's governor is Charles Will...,[William Henry Harrison] is the office holder ...,SELECT DISTINCT ?uri WHERE {?uri <http://dbped...
4,2308,List all the faiths that British Columbian pol...,The religions of the British Columbia politici...,SELECT DISTINCT ?uri WHERE { ?x <http://dbpedi...
...,...,...,...,...
995,1920,How many religion's people have died in Malaysia?,There are [30] religions of the people who die...,SELECT DISTINCT COUNT(?uri) WHERE { ?x <http:/...
996,3336,Where are the National Academy Museum and Scho...,The resting places of the people awarded from ...,SELECT DISTINCT ?uri WHERE { ?x <http://dbpedi...
997,3572,Which uni did Joshua A. Siegel attend ?,The universities Joshua A. Siegel attended are...,SELECT DISTINCT ?uri WHERE { <http://dbpedia.o...
998,1728,What is the base currency of Benelux?,[Euro] is the base currency of Benelux.,SELECT DISTINCT ?uri WHERE { <http://dbpedia.o...


In [10]:
import re
import pandas as pd
def query_fix(query: str) -> str:
    # print(query)
    query = re.sub("<[^>]*\/[^>]*[\/#]", "<", query)
    # query = query.replace(">", "")
    return query
    # print(query)
    
def generate_query_dataset(data_frame):
    data = []
    for idx, row in data_frame.iterrows():
        data.append([row["question"], query_fix(row["query"])])
    
    return pd.DataFrame(data, columns=["question", "query"])

In [11]:
query_dataset = generate_query_dataset(df)
query_dataset

,question,query
0,Count the number of people became famous for w...,SELECT DISTINCT COUNT(?uri) WHERE { ?x <comman...
1,Which location city of Denver Broncos is the p...,SELECT DISTINCT ?uri WHERE { <Denver_Broncos> ...
2,Which people are known for appearing on the te...,SELECT DISTINCT ?uri WHERE {?uri <knownFor> <D...
3,Which office holder's governor is Charles Will...,SELECT DISTINCT ?uri WHERE {?uri <governor> <C...
4,List all the faiths that British Columbian pol...,SELECT DISTINCT ?uri WHERE { ?x <residence> <B...
...,...,...
995,How many religion's people have died in Malaysia?,SELECT DISTINCT COUNT(?uri) WHERE { ?x <deathP...
996,Where are the National Academy Museum and Scho...,SELECT DISTINCT ?uri WHERE { ?x <award> <Natio...
997,Which uni did Joshua A. Siegel attend ?,SELECT DISTINCT ?uri WHERE { <Joshua_A._Siegel...
998,What is the base currency of Benelux?,SELECT DISTINCT ?uri WHERE { <Benelux> <curren...


In [12]:
def get_output(string:str):
    s_index = string.find('[') #stores the index of a substring or char
    e_index = string.find(']')
    return string[s_index+1:e_index]

def get_answer(string: str):
    # should this be done?
    return string.replace('[',"").replace(']', "")

def generate_answer_dataset(data_frame):
    data = []
    for idx, row in data_frame.iterrows():
        data.append([row["question"], get_output(row["verbalized_answer"]), get_answer(row["verbalized_answer"])])
    
    return pd.DataFrame(data, columns=["question", "output", "answer"])

In [13]:
answer_dataset=generate_answer_dataset(df)

In [14]:
query_dataset.to_csv(f"QueryGen/data/vquanda_Q_{target_dataset}.csv", sep='\t', encoding='utf-8', index=False, header=True)
answer_dataset.to_csv(f"AnswerGen/data/vquanda_A_{target_dataset}.csv", sep='\t', encoding='utf-8', index=False, header=True)